## Set up dependencies.

In [1]:
!pip install transformers torch torchinfo tqdm

In [2]:
from pathlib import Path
from tqdm import tqdm
import os
from datasets import load_dataset, Dataset, IterableDataset

## Download the Dolma v1.6_sample dataset (~16GB)

In [3]:
data = load_dataset('allenai/dolma', name='v1_6-sample', trust_remote_code=True, streaming=True, split="train")

## Load phi-2 and set up a method that gets the middle layer's activations.

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)

phi = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.float16, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-07-25 17:36:31.821871: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 17:36:31.859638: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Suppress warnings about tokenising long passages. 
# We're going to handle splitting the text ourselves.
import sys
tokenizer.model_max_length = sys.maxsize

In [6]:
def tokenize(text):
    return tokenizer(text, return_tensors="pt", return_attention_mask=False)['input_ids']

In [7]:
@torch.no_grad()
def get_activations(tokens):
    """
    Takes a tensor of shape e.g. (B, T) containing a batch of token strings,
    and outputs a tensor of shape (B, T, 2560) containing the activations
    of the middle layer (hidden layer 15) at each token.
    """
    out = phi(tokens, output_hidden_states=True, return_dict=True)
    hidden_states = out.hidden_states
    middle_layer = hidden_states[16]
    return middle_layer.float()

In [8]:
def get_token_blocks(text, block_size):
    """
    Given a text, tokenizes it and returns a tensor of shape 
    (N, BS) containing the text as a series of blocks of 
    tokens of size BS.
    N is the number of blocks contained within the text.   
    """    
    block_size = 100
    tokens = tokenize(text).view(-1)
    truncated_length = (len(tokens) // block_size) * block_size
    tokens = tokens[:truncated_length]
    block_tensor = tokens.view(-1, block_size)
    blocks = torch.split(block_tensor, 1, dim=0)    
    return [b.view(-1) for b in blocks]

In [9]:
def all_blocks(split, block_size=100):    
    for i, example in enumerate(data):
        is_test_example = i % 10 == 0
        if split == 'train' and is_test_example or split != 'train' and not is_test_example:
            continue
        for block in get_token_blocks(example['text'], block_size=block_size):                        
            yield block

In [10]:
def batch_iterable(iterable, batch_size):
    current_batch = []
    for item in iterable:
        current_batch.append(item)
        if len(current_batch) == batch_size:
            yield current_batch
            current_batch = []


def all_activations(split, block_size=100, minibatch_size=5, start_block=0, parallelism=10, workahead_cache_size=50):
    """
    Generator yielding minibatches of training examples.
    Each example is returned as a tuple (tokens, activations).
     - tokens is a tensor of dimention (MB, BS) where MB 
       is the specified minibatch size, and BS is the block size.
     - activations is a tensor of dimension (MB, BS, 2560)
    """    
    global phi
    assert workahead_cache_size % parallelism == 0
    assert workahead_cache_size % minibatch_size == 0
    assert parallelism % minibatch_size == 0
    blocks = islice(all_blocks(split, block_size=block_size), start_block, None)
    for block_megabatch in batch_iterable(blocks, workahead_cache_size):  
        phi = phi.to("cuda")
        minibatches = []        
        for inputs in torch.stack(block_megabatch).split(parallelism):            
            activations = get_activations(inputs)      
            minibatch_inputs = inputs.split(minibatch_size)
            minibatch_activations = activations.split(minibatch_size)
            minibatches.extend(zip(minibatch_inputs, minibatch_activations))
        phi = phi.to("cpu")
        
        for minibatch_input, minibatch_activation in minibatches:
            yield minibatch_input, minibatch_activation

## Now define our autoencoder

In [11]:
class SparseAutoencoder(torch.nn.Module):
    def __init__(self, activation_dimension=2560, inner_dimension=100_000):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(activation_dimension, inner_dimension),
            torch.nn.ReLU()
        )
        self.decoder = torch.nn.Linear(inner_dimension, activation_dimension)

    def forward(self, activations):
        encoded = self.encoder(activations)
        decoded = self.decoder(encoded)
        return decoded, encoded  # Return encoded too, as it's used in the loss fn.

In [12]:
if Path('sae.pt').exists():
    sae = torch.load('sae.pt').to(device)
    is_trained = True
else:
    sae = SparseAutoencoder()
    is_trained = False

In [13]:
from torch.nn import functional as F
def calculate_loss(sae_model, prediction, target, feature_activations, lamb=5):
    weight_norms = torch.norm(sae_model.decoder.weight, dim=0, p=2).view(-1)
    feature_sizes = torch.abs(feature_activations)
    sparsity_loss = lamb * torch.sum(weight_norms * feature_sizes)

    prediction_loss = F.mse_loss(prediction, target, reduction='sum')

    total_loss = prediction_loss + sparsity_loss
    
    return total_loss, prediction_loss, sparsity_loss    

In [14]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t, r, a, f)

15655829504 7822376960 7616183296 206193664


## Train our autoencoder on 1M tokens' worth of activations.

In [15]:
from itertools import islice

@torch.no_grad()
def get_eval_loss(sae_model, minibatches, tokens_per_minibatch):
    losses = []
    sparsities = []
    accuracies = []
    for tokens, activations in minibatches:
        decoded, encoded = sae_model(activations)
        loss, accuracy, sparsity = calculate_loss(sae_model, decoded, activations, encoded)
        losses.append(loss.item() / tokens_per_minibatch)
        accuracies.append(accuracy.item() / tokens_per_minibatch)        
        sparsities.append(sparsity.item() / tokens_per_minibatch)        
    return sum(losses) / len(losses), sum(accuracies) / len(accuracies), sum(sparsities) / len(sparsities)

def train(sae_model, num_tokens=1000000, minibatch_size=5, block_size=100, loss_history=100, adam_beta1=0.9, adam_beta2=0.999, lr=5E-5, eval_tokens=1000, show_eval_token_interval=25000, start_block=0):            
    optimizer = torch.optim.AdamW(sae_model.parameters(), lr=lr, betas=(adam_beta1, adam_beta2))
    tokens_per_minibatch = minibatch_size * block_size
    num_minibatches = num_tokens // tokens_per_minibatch
    eval_minibatches = eval_tokens // tokens_per_minibatch
    losses = []
    sparsities = []
    activations_gen = all_activations('train', minibatch_size=minibatch_size, block_size=block_size + start_block)
    test_activations_gen = all_activations('test', minibatch_size=minibatch_size, block_size=block_size, start_block=1000+start_block)    
    
    training_data = islice(activations_gen, num_minibatches)         
    show_eval_minibatch_interval = show_eval_token_interval // tokens_per_minibatch

    pbar = tqdm(training_data, total=num_minibatches, smoothing=0)
    for idx, (tokens, activations) in enumerate(pbar):        
        decoded, encoded = sae_model(activations)        
        loss, accuracy, sparsity = calculate_loss(sae_model, decoded, activations, encoded)        
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()        
        losses.append(loss.item() / tokens_per_minibatch)
        sparsities.append(sparsity.item() / tokens_per_minibatch)
        if len(losses) > loss_history:
            losses = losses[1:]
            sparsities = sparsities[1:]
        avg_loss = sum(losses) / len(losses)
        avg_sparsity = sum(sparsities) / len(sparsities)
        tokens_seen = (idx + 1) * tokens_per_minibatch
        pbar.set_description(f'Tokens seen: {tokens_seen}; Loss: {avg_loss:.2f}; Sparsity-specific loss: {avg_sparsity:.2f}')
        if (idx + 1) % show_eval_minibatch_interval == 0:
            minibatches = islice(test_activations_gen, eval_minibatches)
            eval_loss, _, eval_sparsity = get_eval_loss(sae_model, minibatches, tokens_per_minibatch)
            print(f'Seen {tokens_seen} tokens')
            print(f'\t - Training data: loss is {avg_loss:.2f}; sparsity loss is {avg_sparsity:.2f}')
            print(f'\t - Eval data: loss is {eval_loss:.2f}; sparsity loss is {eval_sparsity:.2f}')            
            torch.save(sae, 'sae.pt')

if not is_trained:
    train(sae)
    is_trained = True
    torch.save(sae, 'sae.pt')

Tokens seen: 25000; Loss: 20933.04; Sparsity-specific loss: 10754.98:   2%|▏         | 49/2000 [00:57<38:03,  1.17s/it] 

Seen 25000 tokens
	 - Training data: loss is 20933.04; sparsity loss is 10754.98
	 - Eval data: loss is 10815.54; sparsity loss is 7070.77


Tokens seen: 50000; Loss: 15219.29; Sparsity-specific loss: 8368.90:   5%|▍         | 99/2000 [01:57<37:27,  1.18s/it] 

Seen 50000 tokens
	 - Training data: loss is 15219.29; sparsity loss is 8368.90
	 - Eval data: loss is 8329.01; sparsity loss is 4630.19


Tokens seen: 75000; Loss: 8548.77; Sparsity-specific loss: 5178.10:   7%|▋         | 149/2000 [02:48<34:55,  1.13s/it] 

...

Tokens seen: 975000; Loss: 2205.94; Sparsity-specific loss: 655.70:  97%|█████████▋| 1949/2000 [33:42<00:52,  1.04s/it]

Seen 975000 tokens
	 - Training data: loss is 2205.94; sparsity loss is 655.70
	 - Eval data: loss is 2496.29; sparsity loss is 642.31


Tokens seen: 1000000; Loss: 2247.79; Sparsity-specific loss: 664.02: 100%|█████████▉| 1999/2000 [34:34<00:01,  1.04s/it]

Seen 1000000 tokens
	 - Training data: loss is 2247.79; sparsity loss is 664.02
	 - Eval data: loss is 2584.46; sparsity loss is 657.67


Tokens seen: 1000000; Loss: 2247.79; Sparsity-specific loss: 664.02: 100%|██████████| 2000/2000 [34:37<00:00,  1.04s/it]


In [16]:
from torchinfo import summary

print(summary(phi, input_shape=(5, 100)))

print(summary(sae, input_data=next(all_activations(split='train'))[1][0:1]))

# sae2 = sae.half()
# print(summary(sae2, input_data=next(all_activations(split='train'))[1][0:1].half()))

Layer (type:depth-idx)                                  Param #
PhiForCausalLM                                          --
├─PhiModel: 1-1                                         --
│    └─Embedding: 2-1                                   131,072,000
│    └─Dropout: 2-2                                     --
│    └─ModuleList: 2-3                                  --
│    │    └─PhiDecoderLayer: 3-1                        78,671,360
│    │    └─PhiDecoderLayer: 3-2                        78,671,360
│    │    └─PhiDecoderLayer: 3-3                        78,671,360
│    │    └─PhiDecoderLayer: 3-4                        78,671,360
│    │    └─PhiDecoderLayer: 3-5                        78,671,360
│    │    └─PhiDecoderLayer: 3-6                        78,671,360
│    │    └─PhiDecoderLayer: 3-7                        78,671,360
│    │    └─PhiDecoderLayer: 3-8                        78,671,360
│    │    └─PhiDecoderLayer: 3-9                        78,671,360
│    │    └─PhiDecoderLayer: 

In [17]:
should_train_more = False
if should_train_more:
    train(sae, start_block=500, num_tokens=100000)

## Now let's see how our accuracy and sparsity do compared to a random Autoencoder.

In [18]:
sae2 = SparseAutoencoder()

In [19]:
tokens, activations = next(all_activations('test', minibatch_size=5, block_size=100, start_block=10000))

In [20]:
with torch.no_grad():
    print(f'Testing on minibatch including:\n---\n{tokenizer.decode(tokens[0])}\n---\n')
    trained_decoded, trained_encoded = sae(activations)    
    untrained_decoded, untrained_encoded  = sae2(activations)

Testing on minibatch including:
---
 and such conditions. Now let
us suppose a teacher of genius to obtain the post. He not only teaches
admirably, but he institutes school gardens for the children; he takes
long walks with the boys, and gives them the rudiments of geology. He
is in himself an uplifting moral influence, and introduces the children
into a whole new world of idea and of feeling. The parents are pleased.
I will not say that they are grateful; but they
---



In [24]:
with torch.no_grad():
    trained_total, trained_accuracy, trained_sparsity = calculate_loss(sae, trained_decoded, activations, trained_encoded)
    untrained_total, untrained_accuracy, untrained_sparsity = calculate_loss(sae2, untrained_decoded, activations, trained_encoded)

print('Untrained model:')
print(f'\t- Accuracy loss: {untrained_accuracy / 2500:.1f}')
print(f'\t- Sparsity loss: {untrained_sparsity / 2500:.1f}')
print(f'\t- Total loss: {untrained_total / 2500:.1f}')
print('')
print('Trained model:')
print(f'\t- Accuracy loss: {trained_accuracy / 2500:.1f}')
print(f'\t- Sparsity loss: {trained_sparsity / 2500:.1f}')
print(f'\t- Total loss: {trained_total / 2500:.1f}')

Untrained model:
	- Accuracy loss: 6043.6
	- Sparsity loss: 87.7
	- Total loss: 6131.2

Trained model:
	- Accuracy loss: 358.1
	- Sparsity loss: 121.2
	- Total loss: 479.3
